In [33]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

df = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
data = df[df["Warengruppe_1"] == 1]

#Werktag
df['Werktag'] = (
    df[['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag']].sum(axis=1) > 0
) & (df['Feiertag'] == 0)
# In 0/1 umwandeln
df['Werktag'] = df['Werktag'].astype(int)

# Sicherstellen, dass "Datum" im Datumsformat ist
df['Datum'] = pd.to_datetime(df['Datum'])
# Jahreszeiten zuweisen
df['Frühling'] = df['Datum'].dt.month.isin([3, 4, 5]).astype(int)
df['Sommer'] = df['Datum'].dt.month.isin([6, 7, 8]).astype(int)
df['Herbst'] = df['Datum'].dt.month.isin([9, 10, 11]).astype(int)
df['Winter'] = df['Datum'].dt.month.isin([12, 1, 2]).astype(int)

# Die ersten Zeilen anzeigen, um die Ergebnisse zu prüfen
print(df[['Datum', 'Frühling', 'Sommer', 'Herbst', 'Winter']].head(10))


df = df.drop(columns=["Datum", "Umsatz"])
print(df)


       Datum  Frühling  Sommer  Herbst  Winter
0 2013-07-01         0       1       0       0
1 2013-07-02         0       1       0       0
2 2013-07-03         0       1       0       0
3 2013-07-04         0       1       0       0
4 2013-07-05         0       1       0       0
5 2013-07-06         0       1       0       0
6 2013-07-07         0       1       0       0
7 2013-07-08         0       1       0       0
8 2013-07-09         0       1       0       0
9 2013-07-10         0       1       0       0
      Warengruppe_1  Warengruppe_2  Warengruppe_3  Warengruppe_4  \
0                 1              0              0              0   
1                 1              0              0              0   
2                 1              0              0              0   
3                 1              0              0              0   
4                 1              0              0              0   
...             ...            ...            ...            ...   
9329   

In [34]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Warengruppe_1"].sort_values(ascending=False)
print(correlation_with_target)




Warengruppe_1           1.000000
Sommer                  0.010090
Frühling                0.008377
Temp_Warm               0.007129
Temp_Mild               0.007114
is_holiday              0.005995
Ship                    0.005768
Number_of_ships         0.005125
Markt                   0.003220
Cloud_Clear             0.003218
Cloud_Partly_Cloudy     0.002981
Temp_Hot                0.002871
Silvester               0.002825
KielerWoche             0.002731
VPI                     0.002702
Ostertag                0.002477
Feiertag                0.001652
Wind_Moderate           0.001616
Weather_Good            0.001609
Mittwoch                0.001135
Wind_Strong             0.000953
Weather_Severe          0.000789
Samstag                 0.000496
Weather_Moderate        0.000455
Freitag                 0.000400
Temp_Very_Cold          0.000260
Werktag                 0.000252
Heimspiel               0.000084
Montag                 -0.000015
Dienstag               -0.000086
Donnerstag

In [35]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Warengruppe_1", axis=1)
y = df["Warengruppe_1"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Warengruppe_5           0.054037
Warengruppe_2           0.052109
Warengruppe_4           0.048508
Warengruppe_3           0.047494
Frühling                0.016640
Temp_Mild               0.012527
Cloud_Clear             0.010830
Winter                  0.010203
Weather_Severe          0.009405
Weihnachtsmarkt         0.009179
Warengruppe_6           0.008875
Ship                    0.008189
Temp_Very_Cold          0.006550
Temp_Cold               0.006419
Markt                   0.004292
Freitag                 0.004081
Cloud_Cloudy            0.003065
Weather_Moderate        0.003020
Number_of_ships         0.002685
Wind_Moderate           0.002632
Donnerstag              0.001967
Heimspiel               0.001920
Temp_Hot                0.001702
Sommer                  0.001652
KielerWoche             0.000848
VPI                     0.000731
Temp_Warm               0.000000
Samstag                 0.000000
Mittwoch                0.000000
Montag                  0.000000
Dienstag  

In [36]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Wind_Light', 'Wind_Moderate', 'Number_of_ships', 'Ship']


In [37]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Warengruppe_1", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                 Feature        VIF
0          Warengruppe_2   1.610242
1          Warengruppe_3   1.610242
2          Warengruppe_4   1.598393
3          Warengruppe_5   1.610242
4          Warengruppe_6   1.218217
5         Temp_Very_Cold        inf
6              Temp_Cold        inf
7              Temp_Mild        inf
8              Temp_Warm        inf
9               Temp_Hot        inf
10           Cloud_Clear        inf
11   Cloud_Partly_Cloudy        inf
12          Cloud_Cloudy        inf
13            Wind_Light        inf
14         Wind_Moderate        inf
15           Wind_Strong        inf
16          Weather_Good        inf
17  Weather_Light_Issues        inf
18      Weather_Moderate        inf
19        Weather_Severe        inf
20           KielerWoche   1.163835
21                Montag        inf
22              Dienstag        inf
23              Mittwoch        inf
24            Donnerstag        inf
25               Freitag        inf
26               Samstag    

In [43]:
# suche miteinander korrelierrende features
df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])

KeyError: "['Warengruppe_1', 'Warengruppe_2', 'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'] not found in axis"

In [47]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Weather_Moderate', 'Weather_Light_Issues'), ('Winter', 'Temp_Mild'), ('Werktag', 'Sonntag'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Heimspiel', 'Samstag'), ('Temp_Mild', 'Temp_Cold'), ('Sommer', 'Temp_Cold'), ('Wind_Moderate', 'Wind_Light'), ('Weather_Light_Issues', 'Weather_Good'), ('Sommer', 'Temp_Warm'), ('Wind_Strong', 'Wind_Moderate'), ('Winter', 'Temp_Cold'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Ship', 'Number_of_ships'), ('Werktag', 'Samstag')}
